In [1]:
from keras.utils.data_utils import get_file

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length =', len(text) )

corpus length = 600901


In [3]:
text[:200]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrib'

In [4]:
chars = sorted(list(set(text))) 
vocab_size = len(chars) + 1
print('vocab length =', vocab_size)

vocab length = 60


Padding data 

In [5]:
chars.insert(0,"\0")

Creating word to indices and backwards

In [6]:
words_indices = dict( (c,i) for i,c in enumerate(chars))
indices_words = dict( (i,c) for i,c in enumerate(chars))

convert all the words into indices

In [7]:
idxs = [ words_indices[char ] for char in text ]

## Preprocessing data
* making sentences of each length 40 chars

In [8]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idxs) - maxlen+1):
    sentences.append(idxs[i: i + maxlen])
    next_chars.append(idxs[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600862


In [9]:
import numpy as np

In [10]:
sentences = np.concatenate( [[np.array(o)] for o in sentences[:-2] ] )
next_chars = np.concatenate( [[np.array(o)] for o in next_chars[:-2]] )

In [11]:
n_fac = 24

In [12]:
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding, Dropout, TimeDistributed, Activation

### Obserevations:
* the gpu load will be around 30% (which should be improved by keras)
* the gpu memory load is high


In [17]:
gener = Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM (50, input_dim=n_fac, return_sequences=True, dropout_U=0.2, dropout_W=0.2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax') )
])

C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Chaitra\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, input_shape=(None, 24), dropout=0.2, return_sequences=True, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
gener.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [29]:
from numpy.random import choice

In [30]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([words_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = gener.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [20]:
gener.fit(sentences, np.expand_dims(next_chars,-1), batch_size=34, epochs=1)

Epoch 1/1
600860/600860 [==============================] - 1979s - loss: 2.1354  


#### LSTM's are harder to train

In [31]:
print_example()

ethics is a basic foundation of all that or peceratity, he vere enconsul insemtudy of care and sesivasing in is endour comparen and what it bleally and adwhick feomming befidire has all daisore it pank very is lighe gor
expar by the resention and to of greatire, the ristion of roge. everyselves well phisuore haken and
of racal, the nelbition, the ropine mee.


* The sentences doesnt make sense. nor the words.
* Let's train the model little bit more 

In [32]:
gener.fit(sentences, np.expand_dims(next_chars,-1), batch_size=34, epochs=2)

Epoch 1/2
600860/600860 [==============================] - 1991s - loss: 1.9757  
Epoch 2/2
600860/600860 [==============================] - 2007s - loss: 1.9486  


In [33]:
print_example()

ethics is a basic foundation of all that one mored some and natures which inconquents men "truth the dooely, such all cerpan to his mode
to in contelling
and
sitimined and
its ame
strarnd and to that mean? new, thatside, "mon plaikarity,"--be their was as for manitide
of grast moralingal rangel moran selusion in the preligst in. ger
and gregher mulkt the exi


    The above paragraph is more understable than before. 
    LSTM's are train to tough. 